# Laboratorium 3: Szyfr Multiplikatywny i Afiniczny

## Setup - Import bibliotek

In [ ]:
from src.crypto_math import gcd, get_mod_inverse
from src.multiplicative_cipher import multiplicative_encrypt, multiplicative_decrypt
from src.defs import SYMBOLS_PL

---
## Zadanie 1: Odwrotność modularna

**Cel**: Implementacja funkcji matematycznych potrzebnych do szyfrów

### 1.1 Największy wspólny dzielnik (GCD) - Algorytm Euklidesa

In [ ]:
# Test funkcji gcd
test_cases = [
    (48, 18),
    (100, 50),
    (17, 13),
    (270, 192),
]

for a, b in test_cases:
    result = gcd(a, b)
    print(f"gcd({a}, {b}) = {result}")

### 1.2 Odwrotność modularna - Rozszerzony algorytm Euklidesa

**Definicja**: Odwrotność modularna liczby `a` modulo `m` to taka liczba `x`, że:
```
(a * x) mod m = 1
```

Odwrotność istnieje tylko gdy `gcd(a, m) = 1`

In [ ]:
symb_len = len(SYMBOLS_PL)
print(f"Długość alfabetu: {symb_len}\n")

test_values = [3, 5, 7, 11, 13, 17]

for a in test_values:
    if gcd(a, symb_len) == 1:
        inv = get_mod_inverse(a, symb_len)
        # Weryfikacja: (a * inv) mod m powinno dać 1
        verification = (a * inv) % symb_len
        print(f"a={a:2d}, odwrotność={inv:3d}, weryfikacja: ({a}*{inv}) mod {symb_len} = {verification} ✓")
    else:
        print(f"a={a:2d}, brak odwrotności (gcd({a}, {symb_len}) != 1) ✗")

---
## Zadanie 2: Szyfr multiplikatywny

**Zasada**: 
- Szyfrowanie: `C = (M * key) mod alphabet_length`
- Deszyfrowanie: `M = (C * key_inverse) mod alphabet_length`

**Warunek**: Klucz musi być względnie pierwszy z długością alfabetu (`gcd(key, len) = 1`)

**Implementacja**: Zobacz `src/multiplicative_cipher.py`

In [ ]:
# Sprawdzenie jakie klucze są dozwolone
symb_len = len(SYMBOLS_PL)
valid_keys = [k for k in range(2, symb_len) if gcd(k, symb_len) == 1]

print(f"Długość alfabetu: {symb_len}")
print(f"Liczba dozwolonych kluczy: {len(valid_keys)}")
print(f"\nPierwsze 20 dozwolonych kluczy: {valid_keys[:20]}")
print(f"Ostatnie 20 dozwolonych kluczy: {valid_keys[-20:]}")

In [ ]:
# Szyfrowanie
message = "Witaj świecie!"
key = 13  # musi być względnie pierwszy z len(SYMBOLS_PL)

if gcd(key, symb_len) != 1:
    print(f"Niepoprawny klucz, gcd({key}, {symb_len}) = {gcd(key, symb_len)}")
else:
    encrypted = multiplicative_encrypt(message, key)
    print(f"Tekst jawny: {message}")
    print(f"Klucz: {key}")
    print(f"Zaszyfrowane: {encrypted}")

In [ ]:
# Deszyfrowanie
cipher = encrypted
decrypted = multiplicative_decrypt(cipher, key)

print(f"Zaszyfrowane: {cipher}")
print(f"Klucz: {key}")
print(f"Odszyfrowane: {decrypted}")
print(f"\nPoprawność: {decrypted == message}")

---
## Zadanie 3: Szyfr afiniczny

**Zasada**: Połączenie szyfru multiplikatywnego i addytywnego (Cezara)
- Szyfrowanie: `C = (M * keyA + keyB) mod alphabet_length`
- Deszyfrowanie: `M = ((C - keyB) * keyA_inverse) mod alphabet_length`

**Warunek**: `keyA` musi być względnie pierwszy z długością alfabetu

**Liczba możliwych kluczy**: 
- `keyA`: liczba względnie pierwszych z `len(alphabet)` (funkcja φ Eulera)
- `keyB`: dowolna liczba od 0 do `len(alphabet)-1`
- Razem: `φ(len) * len` możliwości

In [ ]:
from src.affine_cipher import affine_encrypt, affine_decrypt

symb_len = len(SYMBOLS_PL)
num_valid_keyA = len([k for k in range(1, symb_len) if gcd(k, symb_len) == 1])
num_valid_keyB = symb_len
total_keys = num_valid_keyA * num_valid_keyB

print(f"Długość alfabetu: {symb_len}")
print(f"Możliwe wartości keyA (względnie pierwsze): {num_valid_keyA}")
print(f"Możliwe wartości keyB (dowolne 0-{symb_len-1}): {num_valid_keyB}")
print(f"\nCałkowita liczba możliwych kluczy: {total_keys:,}")

print("\n⏳ Do zaimplementowania: affine_cipher.py")

In [ ]:
# Przykład użycia (po zaimplementowaniu)
# message = "Witaj świecie!"
# keyA = 17  # musi być względnie pierwszy z len(SYMBOLS_PL)
# keyB = 5   # dowolna liczba

# if gcd(keyA, symb_len) != 1:
#     print(f"❌ Niepoprawny klucz keyA={keyA}!")
# else:
#     encrypted = affine_encrypt(message, keyA, keyB)
#     decrypted = affine_decrypt(encrypted, keyA, keyB)
#     
#     print(f"Tekst jawny: {message}")
#     print(f"Klucze: keyA={keyA}, keyB={keyB}")
#     print(f"Zaszyfrowane: {encrypted}")
#     print(f"Odszyfrowane: {decrypted}")
#     print(f"Poprawność: {decrypted == message}")

---
## Zadanie 4: Łamanie szyfru afinicznego (Brute Force)

**Cel**: Złamanie szyfru poprzez sprawdzenie wszystkich możliwych kombinacji kluczy

**Metoda**:
1. Dla każdego możliwego `keyA` (względnie pierwszego z długością alfabetu)
2. Dla każdego możliwego `keyB` (0 do długość-1)
3. Spróbuj odszyfrować
4. Użyj `is_english()` do sprawdzenia czy tekst ma sens
5. Jeśli tak - zapytaj użytkownika czy to poprawna wiadomość

In [ ]:
# TODO: Zaimplementuj w pliku src/affine_cipher_crack.py
# from affine_cipher_crack import crack_affine_cipher
# from detect_english import is_english

# def crack_affine_cipher(cipher):
#     """
#     Łamie szyfr afiniczny metodą brute force
#     """
#     symb_len = len(SYMBOLS_PL)
#     attempts = 0
#     
#     print(f"Rozpoczynam łamanie szyfru afinicznego...")
#     print(f"Będę sprawdzać {total_keys:,} możliwych kombinacji kluczy\n")
#     
#     for keyA in range(1, symb_len):
#         # Pomiń niepoprawne wartości keyA
#         if gcd(keyA, symb_len) != 1:
#             continue
#         
#         for keyB in range(symb_len):
#             attempts += 1
#             
#             decrypted = affine_decrypt(cipher, keyA, keyB)
#             
#             # Sprawdź czy wygląda jak angielski tekst
#             if is_english(decrypted):
#                 print(f"\n{'='*80}")
#                 print(f"Potencjalne rozwiązanie #{attempts}:")
#                 print(f"keyA={keyA}, keyB={keyB}")
#                 print(f"Tekst: {decrypted[:200]}...")
#                 print("="*80)
#                 
#                 response = input("Czy to poprawna wiadomość? (t/n): ")
#                 if response.lower() == 't':
#                     print(f"\n✅ Szyfr złamany!")
#                     return keyA, keyB, decrypted
#     
#     print(f"\n❌ Nie udało się złamać szyfru po sprawdzeniu {attempts} kombinacji")
#     return None, None, None

print("⏳ Do zaimplementowania: affine_cipher_crack.py")
print("⏳ Wymaga: affine_cipher.py oraz detect_english.py")